# Calculation of Various Diversity Metrics

#### Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications.inception_v3 import decode_predictions
import torch
from torchvision import models, transforms
from torch.nn import functional as F
from scipy import linalg
import numpy as np
from PIL import Image
import os
from skimage.metrics import structural_similarity as ssim
from skimage.io import imread
from skimage.color import rgb2gray
import numpy as np

C:\Users\yoel1\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Functions 

In [2]:
def generate_paths(str1):
    path_list = []
    for i in range((20)):
        current_string = str1.replace("$$$", str(i+1))
        path_list.append(current_string)
    return path_list

# Diversity Score Functions

def extract_features(image_paths, model):
    features = []
    for img_path in image_paths:
        img = image.load_img(img_path, target_size=(512, 512))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        feature = model.predict(img)
        feature = feature / np.linalg.norm(feature)
        features.append(feature)
    return np.array(features)


def calculate_diversity_score(features):
    n = len(features)
    distances = []
    for i in range(n):
        for j in range(i+1, n):
            distance = np.linalg.norm(features[i] - features[j])
            distances.append(distance)
    return np.mean(distances)



#### Inception Score Function
def calculate_inception_score(images, splits=10):
    processed_images = []
    for img in images:
        img = img.resize((299, 299))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        processed_images.append(img_array) 
    processed_images = np.vstack(processed_images)
    preds = model.predict(processed_images)
    preds = preds / np.linalg.norm(preds)
    scores = []
    for i in range(splits):
        split_preds = preds[(i * len(preds) // splits):((i + 1) * len(preds) // splits)]
        p_yx = np.mean(split_preds, axis=0)
        kl_div = np.sum(split_preds * (np.log(split_preds + 1e-16) - np.log(p_yx + 1e-16)), axis=1)
        scores.append(np.exp(np.mean(kl_div)))
    return np.mean(scores)



#### FID Score Functions


def get_activations(images, model, batch_size=32, dims=2048, cuda=False):
    model.eval()
    activations = []
    with torch.no_grad():
        for i in range(0, len(images), batch_size):
            batch = images[i:i + batch_size]
            batch = torch.stack([transform(img) for img in batch])
            if cuda:
                batch = batch.cuda()
            pred = model(batch)
            activations.append(pred.cpu().numpy())
    activations = np.concatenate(activations, axis=0)
    return activations


def calculate_fid(real_activations, generated_activations):
    mu_r = np.mean(real_activations, axis=0)
    sigma_r = np.cov(real_activations, rowvar=False)
    mu_g = np.mean(generated_activations, axis=0)
    sigma_g = np.cov(generated_activations, rowvar=False)
    diff = mu_r - mu_g
    covmean, _ = linalg.sqrtm(sigma_r.dot(sigma_g), disp=False)
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = diff.dot(diff) + np.trace(sigma_r + sigma_g - 2 * covmean)
    return fid


def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images



### Intra-Dataset Similarity (MS-SSIM) Functions
def load_images_from_directory(directory_path):
    images = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(directory_path, filename)
            image = imread(image_path)
            images.append(image)
    return images

def compute_ms_ssim(image1, image2):
    image1_gray = rgb2gray(image1)
    image2_gray = rgb2gray(image2)
    ms_ssim_score = ssim(image1_gray, image2_gray, multichannel=False, gaussian_weights=True, use_sample_covariance=False)
    return ms_ssim_score

def compute_dataset_ms_ssim(images):
    n = len(images)
    ms_ssim_scores = []
    for i in range(n):
        for j in range(i + 1, n):
            score = compute_ms_ssim(images[i], images[j])
            ms_ssim_scores.append(score)
    return np.mean(ms_ssim_scores)

def main(directory_path):
    images = load_images_from_directory(directory_path)
    avg_ms_ssim = compute_dataset_ms_ssim(images)
    print(f'Average MS-SSIM: {avg_ms_ssim}')
    
    
    
    
def calculate_fid_test(real,generated):
    inception = models.inception_v3(pretrained=True, transform_input=False)
    inception.fc = torch.nn.Identity()
    inception.eval()
    transform = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    real_image_dir = real
    generated_image_dir = generated
    real_images = load_images_from_folder(real_image_dir)
    generated_images = load_images_from_folder(generated_image_dir)
    real_activations = get_activations(real_images, inception)
    generated_activations = get_activations(generated_images, inception)
    fid_value = calculate_fid(real_activations, generated_activations)
    print(f"FID: {fid_value}") 
    
    
def calculate_metrics(str1,str2):
    set1 = generate_paths(str1) 
    image_paths = set1
    features = extract_features(image_paths, model)
    diversity_score = calculate_diversity_score(features)
    print(f'Diversity Score: {diversity_score}')
    image_files = set1
    images = [Image.open(img_file) for img_file in image_files]
    inception_score = calculate_inception_score(images)
    print("Inception Score:", inception_score)
    main(str2)
    
    
    
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
model = ResNet50(weights='imagenet',include_top=False, pooling='avg', input_shape=(512, 512, 3))


## SDXS Gustav Klimt - euler normal


In [3]:
loc1 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\SDXS Gustav Klimt - euler normal\\img$$$.png"
loc2 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\SDXS Gustav Klimt - euler normal\\"
calculate_metrics(loc1,loc2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 408ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
Diversity Score: 0.10013632476329803
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Inception Score: 1.0329927


C:\Users\yoel1\AppData\Local\Temp\ipykernel_1420\3593262370.py:110: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ms_ssim_score = ssim(image1_gray, image2_gray, multichannel=False, gaussian_weights=True, use_sample_covariance=False)


Average MS-SSIM: 0.30589985768552475


In [4]:
str1 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\Gustav Klimt\\"
str2 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\SDXS Gustav Klimt - euler normal\\"
calculate_fid_test(str1,str2)

C:\Users\yoel1\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\yoel1\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FID: 347.6374650567194


### SDXS Gustav Klimt - Heun sgm_uniform

In [5]:
loc1 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\SDXS Gustav Klimt - Heun sgm_uniform\\img$$$.png"
loc2 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\SDXS Gustav Klimt - Heun sgm_uniform\\"
calculate_metrics(loc1,loc2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
Diversity Score: 0.08305198699235916
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Inception Score: 1.0258796


C:\Users\yoel1\AppData\Local\Temp\ipykernel_1420\3593262370.py:110: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ms_ssim_score = ssim(image1_gray, image2_gray, multichannel=False, gaussian_weights=True, use_sample_covariance=False)


Average MS-SSIM: 0.3233905285345736


In [6]:
str1 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\Gustav Klimt\\"
str2 =  "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\SDXS Gustav Klimt - Heun sgm_uniform\\"
calculate_fid_test(str1,str2)

FID: 359.2462077014194


### SDXS Gustav Klimt - sdepp_2m karras

In [7]:
loc1 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\SDXS Gustav Klimt - sdepp_2m karras\\img$$$.png"
loc2 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\SDXS Gustav Klimt - sdepp_2m karras\\"
calculate_metrics(loc1,loc2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
Diversity Score: 0.0901665911078453
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Inception Score: 1.0289125


C:\Users\yoel1\AppData\Local\Temp\ipykernel_1420\3593262370.py:110: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ms_ssim_score = ssim(image1_gray, image2_gray, multichannel=False, gaussian_weights=True, use_sample_covariance=False)


Average MS-SSIM: 0.314871693656394


In [8]:
str1 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\Gustav Klimt\\"
str2 =  "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\SDXS Gustav Klimt - sdepp_2m karras\\"
calculate_fid_test(str1,str2)

FID: 369.7304681773497


### First Batch - SDSX API (colab)

In [9]:
loc1 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\SDSX API\\img$$$.png"
loc2 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\SDSX API\\"
calculate_metrics(loc1,loc2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
Diversity Score: 0.10282935947179794
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Inception Score: 1.0486385


C:\Users\yoel1\AppData\Local\Temp\ipykernel_1420\3593262370.py:110: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ms_ssim_score = ssim(image1_gray, image2_gray, multichannel=False, gaussian_weights=True, use_sample_covariance=False)


Average MS-SSIM: 0.23105976338181058


In [10]:
str1 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\Gustav Klimt\\"
str2 =  "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\SDSX API\\"
calculate_fid_test(str1,str2)

FID: 355.4883777112466


### DreamShaper

In [12]:
loc1 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\\Dreamshaper Gustav Klimt final\\img$$$.png"
loc2 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\\Dreamshaper Gustav Klimt final\\"
calculate_metrics(loc1,loc2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
Diversity Score: 0.19623003900051117
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Inception Score: 1.1376936


C:\Users\yoel1\AppData\Local\Temp\ipykernel_1420\3593262370.py:110: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ms_ssim_score = ssim(image1_gray, image2_gray, multichannel=False, gaussian_weights=True, use_sample_covariance=False)


Average MS-SSIM: 0.18792125687102046


In [13]:
str1 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\Gustav Klimt\\"
str2 =  "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\Dreamshaper Gustav Klimt final\\"
calculate_fid_test(str1,str2)

FID: 374.8401933876367


#### Fusion 1

In [14]:
def calculate_metrics(str1,str2):
    set1 = generate_paths(str1) 
    image_paths = set1
    features = extract_features(image_paths, model)
    diversity_score = calculate_diversity_score(features)
    print(f'Diversity Score: {diversity_score}')
    image_files = set1
    images = [Image.open(img_file) for img_file in image_files]
    inception_score = calculate_inception_score(images)
    print("Inception Score:", inception_score)
    main(str2)
    
loc1 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\Fusion 1\\img$$$.png"
loc2 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\Fusion 1"
calculate_metrics(loc1,loc2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
Diversity Score: 0.18728163838386536
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Inception Score: 1.1452467


C:\Users\yoel1\AppData\Local\Temp\ipykernel_1420\3593262370.py:110: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ms_ssim_score = ssim(image1_gray, image2_gray, multichannel=False, gaussian_weights=True, use_sample_covariance=False)


Average MS-SSIM: 0.1350263113736044


In [15]:
str1 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\Gustav Klimt\\"
str2 =  "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\Fusion 1\\"
calculate_fid_test(str1,str2)

FID: 325.39204176513675


### Fusion 2

In [16]:
loc1 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\Fusion 2\\img$$$.png"
loc2 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\Fusion 2"
calculate_metrics(loc1,loc2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
Diversity Score: 0.20014441013336182
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Inception Score: 1.1394771


C:\Users\yoel1\AppData\Local\Temp\ipykernel_1420\3593262370.py:110: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ms_ssim_score = ssim(image1_gray, image2_gray, multichannel=False, gaussian_weights=True, use_sample_covariance=False)


Average MS-SSIM: 0.15206815821772876


In [17]:
str1 = "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\Gustav Klimt\\"
str2 =  "Y:\\Data Science Readings\\Advanced Computational Learning and Data Analysis\\final project\\pictures\\Fusion 2\\"
calculate_fid_test(str1,str2)

FID: 353.66854102186903
